In [10]:
import pandas as pd
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, create_subplot_grid, plot_bar, plot_statistical_strip
from core.s3 import S3AssetManager


In [11]:
notebook_name = "ecuador_ensayo_rehumectante"
s3 = S3AssetManager(notebook_name=notebook_name)
PALETTE =  [ "#1c8074", "#666666","#1a494c", "#94af92", "#666666", "#f9ee77", "#f5ad68", "#c76931"]



In [12]:
raw = s3.read_excel("raw/ecuador/Rehumectacion datos (1).xlsx", skiprows=8, sheet_name="Hoja1")
raw = raw.rename(columns={"Sackoff": "diferencia kilos"})
raw

,producto,rehumectante,humedad,actividad agua,temperatura,programado,producido,sackoff,ganancia,ganancia neta total,otra,2.35,Presio
0,harina,si,10.94,NaN,81.6,NaN,NaN,NaN,NaN,NaN,NaN,9.4,NaN
1,harina,no,10.51,NaN,65.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.544402
2,pellet,si,9.96,0.67,NaN,6000.0,6320.0,320.0,160.0,103.6,17.266667,ROI,1.544402
3,pellet,no,9.50,0.69,NaN,8000.0,8120.0,120.0,60.0,NaN,7.500000,NaN,NaN


In [13]:
raw = raw[raw["producto"].notnull()]

In [14]:
raw.loc[:, "sackoff(%)"] = raw["sackoff"]/raw["programado"]*100
raw

,producto,rehumectante,humedad,actividad agua,temperatura,programado,producido,sackoff,ganancia,ganancia neta total,otra,2.35,Presio,sackoff(%)
0,harina,si,10.94,NaN,81.6,NaN,NaN,NaN,NaN,NaN,NaN,9.4,NaN,NaN
1,harina,no,10.51,NaN,65.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.544402,NaN
2,pellet,si,9.96,0.67,NaN,6000.0,6320.0,320.0,160.0,103.6,17.266667,ROI,1.544402,5.333333
3,pellet,no,9.50,0.69,NaN,8000.0,8120.0,120.0,60.0,NaN,7.500000,NaN,NaN,1.500000


In [15]:
def rehumectante_comparison_plot_styled(
    df: pd.DataFrame,
    group_col: str = "producto",
    condition_col: str = "rehumectante",
    metrics: list[str] = None,
    threshold: float = 0.05,
    decimals: int = 2,
    show_percentage_change: bool = True,
    colors: tuple = ("#1c8074", "#666666"),
    save_path: str | Path | None = None,
    save_format: str = "png",  # o 'svg', 'jpeg', etc.
    width: int = 1200,
    height: int = 600,
) -> go.Figure:
    import plotly.graph_objects as go
    import numpy as np

    if metrics is None:
        metrics = df.select_dtypes(include=np.number).columns.tolist()

    fig = go.Figure()
    products = df[group_col].unique()
    conditions = df[condition_col].unique()

    x_vals = []
    base_vals = []
    labels = []

    for metric in metrics:
        for product in products:
            row = df[df[group_col] == product]
            if row.empty: continue

            values = []
            for cond in conditions:
                value = row[row[condition_col] == cond][metric]
                values.append(value.values[0] if not value.empty else None)

            if any(v is not None for v in values):
                x_vals.append(f"{product} - {metric}")
                base_vals.append(values)
                labels.append((product, metric))

    for i, cond in enumerate(conditions):
        fig.add_trace(go.Bar(
            x=[x for x, _ in labels],
            y=[vals[i] for vals in base_vals],
            name=f"Rehumectante: {cond}",
            marker_color=colors[i],
            text=[f"{v:.2f}" if v is not None else "" for v in [vals[i] for vals in base_vals]],
            textposition="auto",
        ))

    if show_percentage_change:
        for i, (product, metric) in enumerate(labels):
            v_si, v_no = base_vals[i]
            if None not in (v_si, v_no) and v_no != 0:
                pct = (v_si - v_no) / abs(v_no)
                if abs(pct) > threshold:
                    fig.add_annotation(
                        x=f"{product} - {metric}",
                        y=0,  # en la base
                        text=f"Δ {pct:+.{decimals}%}",
                        showarrow=False,
                        font=dict(size=11, color="black"),
                        yanchor="top",
                        yshift=-10  # opcional: para bajarlo un poco más
                    )

    all_y = [v for pair in base_vals for v in pair if v is not None]
    y_max = max(all_y) if all_y else 1
    y_margin = y_max * 0.15
    yaxis_title = metrics[0] if len(metrics) == 1 else "Valor"


    fig.update_layout(
        title=dict(text="Comparación de métricas por producto con y sin rehumectante", x=0.5, font=dict(color="black")),
        xaxis_title="Producto - Métrica",
        yaxis_title="Valor",
        barmode="group",
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        xaxis=dict(
            title="Producto - Métrica",
            title_font=dict(color="black"),
            tickfont_color="black",
            linecolor="black",
            mirror=True,
            showline=True,
            gridcolor="rgba(0,0,0,0.1)"
        ),
        yaxis=dict(
            title=yaxis_title,
            title_font=dict(color="black"),
            tickfont_color="black",
            linecolor="black",
            mirror=True,
            showline=True,
            gridcolor="rgba(0,0,0,0.1)",
            range=[0, y_max + y_margin]
        ),

        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5,
            font=dict(color="black")
        )
    )

    fig.add_shape(type="rect",
                  xref="paper", yref="paper",
                  x0=0, y0=0, x1=1, y1=1,
                  line=dict(color="black", width=1))

    # Guardar como imagen si se proporciona una ruta
    if save_path:
        if save_format == "html":
            fig.write_html(str(save_path), include_plotlyjs="cdn")
            print(f"[✓] HTML interactivo guardado en: {save_path}")
        else:
            fig.write_image(str(save_path), format=save_format, width=width, height=height)
            print(f"[✓] Imagen guardada en: {save_path}")


    return fig


In [203]:
cols = [
    'producto',        'rehumectante',      'actividad agua',
               'temperatura',          'programado',           'producido',
                   'sackoff',            'ganancia', 'ganancia neta total',
                      'otra',
               'pct_sackoff'
]
raw[cols].to_csv("../raw/rehumectante.csv", index=False)

In [17]:
fig = rehumectante_comparison_plot_styled(
    raw, #[raw["producto"]=='pellet'],
    metrics=["humedad",],
    show_percentage_change=True,
    #save_path=f"{ROOT_IMAGEN}/comparacion_rehumectante_humedad.html",  # Aquí la ruta y formato
    save_format="html"
)
fig.show()
s3.save_plotly_html(fig, "comparacion_rehumectante_humedad.html")

In [19]:
fig = rehumectante_comparison_plot_styled(
    raw[raw["producto"]=='pellet'],
    metrics=["actividad agua",],
    show_percentage_change=True,
    #save_path=f"{ROOT_IMAGEN}/comparacion_rehumectante_aw.html",  # Aquí la ruta y formato
    save_format="html"
)
fig.show()
s3.save_plotly_html(fig, "comparacion_rehumectante_aw.html")

In [20]:
fig = rehumectante_comparison_plot_styled(
    raw[raw["producto"]=='pellet'],
    metrics=["sackoff",],
    show_percentage_change=False,
    #save_path=f"{ROOT_IMAGEN}/comparacion_rehumectante_sackoff.html",  # Aquí la ruta y formato
    save_format="html"
)
fig.show()
s3.save_plotly_html(fig, "comparacion_rehumectante_sackoff.html")

In [21]:
fig = rehumectante_comparison_plot_styled(
    raw[raw["producto"]=='pellet'],
    metrics=["sackoff(%)",],
    show_percentage_change=False,
    #save_path=f"{ROOT_IMAGEN}/rehu.html",  # Aquí la ruta y formato
    save_format="html"
)
fig.show()
s3.save_plotly_html(fig, "rehu.html")

In [22]:
fig = rehumectante_comparison_plot_styled(
    raw[raw["producto"]=='pellet'],
    metrics=["programado",],
    show_percentage_change=False,
    #save_path=f"{ROOT_IMAGEN}/comparacion_rehumectante_prod.html",  # Aquí la ruta y formato
    save_format="html"
)
fig.show()
s3.save_plotly_html(fig, "comparacion_rehumectante_prod.html")

In [23]:
fig = rehumectante_comparison_plot_styled(
    raw[raw["producto"]!='pellet'],
    metrics=["temperatura",],
    show_percentage_change=False,
    #save_path=f"{ROOT_IMAGEN}/comparacion_rehumectante_temp.html",  # Aquí la ruta y formato
    save_format="html"
)
fig.show()
s3.save_plotly_html(fig, "comparacion_rehumectante_temp.html")

In [24]:
fig = rehumectante_comparison_plot_styled(
    raw[raw["producto"]=='pellet'],
    metrics=["ganancia",],
    show_percentage_change=False,
   #save_path=f"{ROOT_IMAGEN}/comparacion_rehumectante_ganancia.html",  # Aquí la ruta y formato
    save_format="html"
)
fig.show()
s3.save_plotly_html(fig, "comparacion_rehumectante_ganancia.html")

In [25]:
cols = ['producto',        'rehumectante',             'humedad',
'actividad agua',         'temperatura',          'programado',
     'producido',             'sackoff',            'ganancia',
'ganancia neta total']

In [26]:
dep= raw[cols]

In [27]:
dep["ROI"] = dep["ganancia"] /dep["ganancia neta total"]

/var/folders/1g/77kw2x4j5678s_87_sqc1fpc0000gp/T/ipykernel_56983/3273148165.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [28]:
dep = dep.round(2)
dep.fillna('--', inplace=True)
s3.save_dataframe(dep, "rehumectante_dep.csv")